**Note:** This notebook illustrates the limitations of `qrml` with possible diagnostics, however for further details, please refer to the source code or the paper [[1]](#1).

In [2]:
import qrml
import numpy                as np
import matplotlib.pyplot    as plt

%matplotlib qt

## Limitations

The case of the torus illustrates the main limitations of using `qrml` to identify the quotient contruction of pointcloud surfaces. 

In [3]:
torus = np.loadtxt('../datasets/torus.txt')  # 1000 points
params = {'S1':0.2, 'k':10, 'threshold_var':0.05, 'edge_sen':1, 'k0':100}

S_torus = qrml.Simplex()
S_torus.build_simplex(torus, **params)
S_torus.normal_coords(**params)

Restricted license - for non-production use only - expires 2023-10-25


### Incorrect Boundary

We note that the torus (and more complex surfaces) are contructed with more than a single 1-cycle in the quotient - e.g. in the canonical contruction of the torus, the boundary of the square is identified with a bouquet of two circles. Our quotient algorithm relies on the "naive" projection algorithm in [[2]](#2) being able to assign all the gluing/short-circuit points, which canonically form the quotient, to the boundary of the projection.

In [10]:
S_torus.show_boundary(alpha=1, tol=2, c=torus[:, 0], show_tear_points=True, a=2.5, show_connections=True, show_pointcloud=True, connection_tol=5, **params)

However, from `S_torus.show_boundary` we see that the boundary of our projection only corresponds to a single 1-cycle. We can see from the "tear" points in the pointcloud, which form a bouquet of two circles, that the points making up the other 1-cycle in the torus are projected to opposing sides in the projection, inside the boundary. This results in `show_connections=True` only showing a single pair of glued edges. 

The reason for this lies with the fact that the "naive" projection algorithm preserves the distance of the shortest path, in the 1-skeleton, between the base point and a generic point in the projection of the points on this path. Due to the different positions of the 1-cycles, only one is assigned to the boundary.

Future work is required to modify this projection algorithm to place the majority of "tear" points to the boundary to give the correct boundary to assign quotient identifications to.

### Incorrect Edge Refinements

From looking at the boundary connections in `S_torus.show_boundary` and our parameters `params`, we see that our quotient algorithm will identify the entire boundary as a single short-edge. It will then refine this short-edge by splitting along one of the connections. This connection will be chosen by random as all short-circuit points have one allowable connection.

However, for any choice, at least one of the two new short-edges will have the majority of its boundary connections connected to itself. Hence, when the algorithm comes to compute how short-edges are connected together, it will return an error as this edge will be associated with itself. This is a failure of our edge refinement.

In [12]:
quotient_info = S_torus.plot_quotient(c=torus[:, 0], alpha=1, tol=2, quotient_tol=15, tol1=5, connection_tol=5, alpha0=0.8, show_pointcloud=True, **params)

Short-circuit edges not properly seperated! Try another value of tol1


This issue is partly due to the limitations of the projection algorithms giving an incorrect boundary for the quotient and partly due to how we choose to refine short-edges. Future work is need to address this issue. Possibilities include choosing to refine such edges by another method than along self-connection points, or if a refined short-edge is mainly connected to itself, then we instead take it to be glued to its second most connected short-edge etc.

## Diagnostics

To help with choosing parameters, we have the functions `find_short_and_refined` and `plot_edges` in `qrml` which show how short and non-short edges are choosen by our algorithm and how these edges are refined. Try the below cells to see this in action in how we can salvage a quotient identification from the torus.

In [15]:
edge_info = qrml.find_short_and_refined(S_torus, alpha=1, tol=2, quotient_tol=15, tol1=5)  # quotient_tol = 15
qrml.plot_edges(S_torus, c=torus[:, 0], edge_info=edge_info, alpha0=0.8)

In [16]:
edge_info = qrml.find_short_and_refined(S_torus, alpha=1, tol=2, quotient_tol=5, tol1=5)  # quotient_tol = 5
qrml.plot_edges(S_torus, c=torus[:, 0], edge_info=edge_info, alpha0=0.8)

In [17]:
_ = S_torus.plot_quotient(c=torus[:, 0], alpha=1, tol=2, quotient_tol=5, tol1=5, connection_tol=5, alpha0=0.8, show_pointcloud=True, **params)

#### References

<a id="1">[1]</a> 
TODO

<a id="2">[2]</a> 
Tong, L., Zha, H. Riemannian manifold learning. *IEEE Transactions on Pattern Analysis
and Machine Intelligence* 30.5 (2008): 796-809.